[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/MGP/blob/main/GP/Planejamento_Agregado/q1.ipynb)

# Um Problema de Mix de Produção

$$\max z = 0,3x_1 + 0,25x_2 + 0,2x_3$$
                                            sujeito a:
$$0,22x_1 + 0,52x_2 + 0,74x_3 \leq 9600000$$
$$0,52x_1 + 0,34x_2 + 0,20x_3 \leq 4800000$$
$$0,28x_1 + 0,14x_2 + 0,06x_3 \leq 2200000$$ 
$$x_3 \geq 16x_1$$
$$x_2 \leq 600000$$
$$x_i \geq 0,   \forall i \in \{1,2,3\}$$
  

In [ ]:
#Execute esse bloco caso esteja executando no Google Colab
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

In [ ]:
import pyomo.environ as pe

## Modelando diretamente

In [ ]:
M=pe.ConcreteModel()
M.x = pe.Var([1,2,3], within=pe.NonNegativeReals)
M.z = pe.Objective(rule = 0.3*M.x[1] + 0.25*M.x[2] + 0.2*M.x[3], sense=pe.maximize)
M.R1 = pe.Constraint(rule = 0.22*M.x[1]+ 0.52 * M.x[2] + 0.74*M.x[3] <= 9600000)
M.R2 = pe.Constraint(rule = 0.5*M.x[1]+ 0.34 * M.x[2] + 0.2*M.x[3] <= 4800000)
M.R3 = pe.Constraint(rule = 0.28*M.x[1]+ 0.14 * M.x[2] + 0.06*M.x[3] <= 2200000)
M.R4 = pe.Constraint(rule = M.x[3] >= 16*M.x[1])
M.R5 = pe.Constraint(rule = M.x[2] <= 600000)
pe.SolverFactory('glpk').solve(M)
print(f"z= {pe.value(M.z)}")
for k in M.x.keys(): 
    print (f"x_{k} = {pe.value(M.x[k])}")

## Modelando a partir dos dados

As vezes pode ser importante inicializar os parâmetros no modelo. Assim, o modelo...

In [ ]:
import pandas as pd

In [ ]:
dados = pd.DataFrame({'pura':[0.22, 0.52, 0.74],
                      'octana':[0.5, 0.34, 0.2],
                      'aditivo': [0.28, 0.14, 0.06]},
                      index = ['verde', 'azul', 'comum'])
dados


In [ ]:
print(list(dados.index))
print(list(dados.columns))

In [ ]:
comp_dict = dados.stack().to_dict()
comp_dict

In [ ]:
capacidades = {'pura':9600000, 'octana': 4800000, 'aditivo': 2200000}
lucros = {'verde':0.3, 'azul': 0.25, 'comum': 0.2}

In [ ]:
M = pe.ConcreteModel()

In [ ]:
# Conjuntos
M.tipos_prod = pe.Set(initialize = list(dados.index)) #tipos de produto que podem ser produzidos
M.mp = pe.Set(initialize = list(dados.columns)) #matérias-primas para a produção


In [ ]:
#Parâmetros
M.comp = pe.Param(M.tipos_prod, M.mp, initialize=comp_dict) #composição dos produtos
M.capacidades = pe.Param(M.mp, initialize= capacidades) #capacidades
M.lucros = pe.Param(M.tipos_prod, initialize= lucros) #lucros
M.capacidade_azul = pe.Param(initialize = 600000)


In [ ]:
M.x = pe.Var(M.tipos_prod, within = pe.NonNegativeReals)

In [ ]:
def Lucro(M):
    return sum(M.x[i] * M.lucros[i] for i in M.tipos_prod)

M.z = pe.Objective(rule = Lucro, sense = pe.maximize)

In [ ]:
M.R_capacidades = pe.ConstraintList()

for j in M.mp:
    M.R_capacidades.add (expr = sum(M.x[i] * M.comp[i, j] for i in M.tipos_prod) <= M.capacidades[j])

M.R2 = pe.Constraint(expr = M.x['comum'] >= 16 * M.x['verde'])

M.R3 = pe.Constraint(expr = M.x['azul'] <= M.capacidade_azul)

In [ ]:
pe.SolverFactory('glpk').solve(M)

In [ ]:
print(f"z= {pe.value(M.z)}")
for k in M.x.keys(): 
    print (f"x_{k} = {pe.value(M.x[k])}")
